Πρόσφατες επιθέσεις, όπως εκείνη στον πάροχο DNS Dyn (<a href="https://en.wikipedia.org/wiki/DDoS_attack_on_Dyn">wiki</a>) αποδεικνύουν ότι ένα από τα σημαντικότερα προβλήματα που αντιμετωπίζει το σύγχρονο Διαδίκτυο είναι εκείνο των botnets. Σε αυτά, ένας επιτιθέμενος συγκεντρώνει την υπολογιστική ισχύ που του είναι απαραίτητη για την εκδήλωση επιθέσεων DDoS ή/και άλλων κακόβουλων δραστηριοτήτων εγκαθιστώντας λογισμικό σε μεγάλο πλήθος από υπολογιστές (bots) που έχουν κενά ασφαλείας (π.χ. συσκευές Internet of Things - IoT).

Οι μολυσμένοι υπολογιστές (bots) διατηρούν διαύλους επικοινωνίας με το διαχειριστή του botnet (Command & Control Server) με σκοπό να λαμβάνουν εντολές και να αποστέλλουν πληροφορίες. Για το σκοπό αυτό εκμεταλλεύονται καθιερωμένα πρωτόκολλα, όπως το DNS με την παραγωγή μεγάλου πλήθους από domain names μέσω Domain Generation Algorithms (DGA's) που αλλάζουν διαρκώς για την επικοινωνία του bot με το διαχειριστή του, ώστε να αποφεύγεται ο εντοπισμός του Command & Control Server.

Τα ονόματα DNS που χρησιμοποιούνται από αλγορίθμους DGA μπορεί να είναι είτε τυχαία αλφαριθμητικά (π.χ. asdfasjkdfh8oawher8has.com) ή συνδυασμοί τυχαίων λέξεων που έχουν ληφθεί από κάποιο λεξικό (π.χ. school-doctor.com). Χρησιμοποιείται ένας μεγάλος αριθμός από τέτοια ονόματα, η πλειοψηφία των οποίων δεν έχουν κάποια αντιστοίχιση σε διεύθυνση IP και στοχεύουν στην απόκρυψη του Command & Control Server, επειδή οι αμυνόμενοι καλούνται να ελέγξουν κάθε ένα από τα ονόματα που παρατηρούν στο δίκτυό τους, σπαταλώντας χρόνο και πόρους. Επιπρόσθετα, η διεύθυνση IP του Command & Control Server αλλάζει πολύ συχνά (πολλές φορές σε μία μέρα), ώστε να αποφεύγεται ο εντοπισμός του ακόμα και όταν εντοπίζονται τα ονόματα DGA που οδήγησαν σε αυτόν.

<img src="https://raw.githubusercontent.com/nkostopoulos/StochasticsLabPublic/master/lab12/dga.png"></img>

In [1]:
import numpy
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import matplotlib.pyplot as plt
import random

def add_word_distinct_chars(string, distinct_chars):
    for char in string:
        distinct_chars.add(char)
    return distinct_chars

def find_max_len(string, max_len):
    string_length = len(string)
    if string_length > max_len:
        max_len = string_length
    return max_len

def load_data(filename):
    dataset = []
    distinct_chars = set()
    max_len = 0
    currentIndex = 0
    diverse_labels = dict()
    with open(filename, "r") as fdr:
        for line in fdr:
            line = line.strip()
            general, label, name = line.split(",")
            name = name.split(".")[0]
            if label not in diverse_labels.keys():
                diverse_labels[label] = currentIndex
                currentIndex += 1
            distinct_chars = add_word_distinct_chars(name, distinct_chars)
            max_len = find_max_len(name, max_len)
            temp_list = []
            temp_list.append(name)
            temp_list.append(label)
            dataset.append(temp_list)
    random.shuffle(dataset)
    return dataset, distinct_chars, max_len, diverse_labels

def assign_index(chars):
    features = {}
    for index, char in enumerate(chars):
        features[char] = index
    return features

def convert_dataset_and_tokenize(dataset, features, max_len):
    for item_no, example in enumerate(dataset):
        name = example[0]
        label = example[1]
        tokenized = []
        padding_needed = max_len - len(name)
        for index in range(padding_needed):
            tokenized.append(0)
        for char in name:
            token = features[char]
            tokenized.append(token)
        example[0] = tokenized
        dataset[item_no] = example
    return dataset

def split_examples_labels(dataset):
    examples = [entry[0] for entry in dataset]
    labels = [entry[1] for entry in dataset]
    return examples, labels

def convert_labels_to_int(labels, diverse_labels):
    for index, label in enumerate(labels):
        labels[index] = diverse_labels[label]
    return labels

def build_model(max_features, max_len):
    model = Sequential()
    model.add(Embedding(max_features, 128, input_length = max_len))
    model.add(LSTM(128))
    model.add(Dropout(0.5))
    model.add(Dense(26, activation = 'softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
    return model

def return_fold(examples, labels_int, num_folds, current_fold):
    interval = len(examples) // num_folds
    current_start = (current_fold - 1) * interval
    current_end = current_fold * interval
    X_test = examples[current_start:current_end]
    y_test = labels_int[current_start:current_end]
    X_train = examples[:current_start]
    y_train = labels_int[:current_start]
    X_train.extend(examples[current_end:])
    y_train.extend(labels_int[current_end:])
    return X_train, y_train, X_test, y_test

def make_predictions(model, X_test):
    y_pred = model.predict(X_test)
    y_pred = numpy.argmax(y_pred, axis = 1)
    return y_pred

def return_confusion_matrix(y_test, y_pred):
    return confusion_matrix(y_test, y_pred)

def return_classification_report(y_test, y_pred, diverse_names):
    target_names = []
    for item in diverse_names.keys():
        target_names.append(item)
    print(classification_report(y_test, y_pred, target_names = target_names))
    return None
    
if __name__ == "__main__":
    dataset, distinct_chars, max_len, diverse_labels = load_data("dga_domains_full.csv")
    dataset = dataset[0:100000]
    max_features = len(distinct_chars) + 1
    features = assign_index(distinct_chars)
    dataset = convert_dataset_and_tokenize(dataset, features, max_len)
    examples, labels = split_examples_labels(dataset)
    labels_int = convert_labels_to_int(labels, diverse_labels)

    acc_per_fold = []
    loss_per_fold = []
    number_of_folds = 5
    for fold in range(1, number_of_folds + 1):
        X_train, y_train, X_test, y_test = return_fold(examples, labels_int, number_of_folds, fold)
        model = build_model(max_features, max_len)
        print("Training for fold: ", fold)
        history = model.fit(X_train, y_train, batch_size = 128, epochs = 5, verbose = 1)
        scores = model.evaluate(X_test, y_test, verbose = 1)
        print(f'Score for fold {fold}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
        acc_per_fold.append(scores[1] * 100)
        loss_per_fold.append(scores[0])
        y_pred = make_predictions(model, X_test)
        print(return_confusion_matrix(y_test, y_pred))
        return_classification_report(y_test, y_pred, diverse_labels)

    # == Provide average scores ==
    print('------------------------------------------------------------------------')
    print('Score per fold')
    for i in range(0, len(acc_per_fold)):
        print('------------------------------------------------------------------------')
        print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
        print('------------------------------------------------------------------------')
        print('Average scores for all folds:')
        print(f'> Accuracy: {numpy.mean(acc_per_fold)} (+- {numpy.std(acc_per_fold)})')
        print(f'> Loss: {numpy.mean(loss_per_fold)}')
        print('------------------------------------------------------------------------')


Training for fold:  1
Epoch 1/5
625/625 [==============================] - 220s 347ms/step - loss: 1.4620 - accuracy: 0.6075
Epoch 2/5
625/625 [==============================] - 246s 393ms/step - loss: 0.9428 - accuracy: 0.7139
Epoch 3/5
625/625 [==============================] - 212s 339ms/step - loss: 0.7906 - accuracy: 0.7553
Epoch 4/5
625/625 [==============================] - 194s 310ms/step - loss: 0.7033 - accuracy: 0.7785
Epoch 5/5
625/625 [==============================] - 27s 39ms/step - loss: 0.5806 - accuracy: 0.8097
Score for fold 1: loss of 0.5805745720863342; accuracy of 80.97000122070312%
[[ 238    0  155    0    1    0    0    4    0    0    0    0    0    0
     0    0    0    0    1    0    0    0    0    0    0    1]
 [   0  398    8    0    0    0    0    0    0    0    0    0    0    1
     0    0    0    1    1    0    0    0    0    6    0    2]
 [  47    2 9657    1   20    4    2   25   42   13   28    1   13    0
    15    6    5    0   19    0   22   38    0

Ο παραπάνω κώδικας επιλύει το πρόβλημα της ταξινόμησης ονομάτων DNS ως προς το αν αυτά τα ονόματα είναι καλόβουλα ή έχουν παραχθεί από Domain Generation Algorithms (DGA's). Συγκεκριμένα, ο κώδικας επιλύει ένα πρόβλημα multi-class classification, όπου τα ονόματα αντιστοιχίζονται είτε στην κατηγορία alexa, εάν είναι καλόβουλα ή στην αντίστοιχα κατηγορία malware στην οποία ανήκουν. Το μοντέλο που έχει χρησιμοποιηθεί για την ταξινόμηση των ονομάτων είναι το LSTM, ενώ για το κατάλληλο validation του μοντέλου έχει χρησιμοποιηθεί η μέθοδος K-fold cross validation με Κ = 5. Τα δεδομένα εκπαίδευσης και αξιολόγησης έχουν ληφθεί από <a href="https://github.com/chrmor/DGA_domains_dataset">εδώ</a> και χρησιμοποιούνται τα πρώτα 100000 ονόματα του αρχείου "dga_domains_full.csv". 

Μελετώντας τον κώδικα και εκτελώντας τον, να απαντήσετε στις ακόλουθες ερωτήσεις:
<ul>
<li>Γιατί το LSTM είναι κατάλληλο μοντέλο για την επίλυση του συγκεκριμένου προβλήματος;</li>
<li>Τι είναι η λίστα ονομάτων Alexa (<a href="https://en.wikipedia.org/wiki/Alexa_Internet">info</a>), ονόματα της οποίας έχουν χρησιμοποιηθεί για την κατηγορία των καλόβουλων ονομάτων;</li>
<li>Να διαλέξετε δύο οικογένειες malware από την κατηγορία dga, δηλαδή την κατηγορία με τα κακόβουλα ονόματα που αντιστοιχούν σε malware. Να αναζητήσετε τι προβλήματα δημιουργούν τα malware αυτά, π.χ. υποκλοπή τραπεζικών κωδικών, έναρξη επιθέσεων DDoS, κλπ.</li>
<li>Να περιγράψετε σύντομα τα βήματα που ακολουθεί το παραπάνω πρόγραμμα για την επίλυση του προβλήματος.</li>
<li>Ποιος είναι ο ρόλος του embedding layer και τι παραμέτρους δέχεται;</li>
<li>Να αναφέρετε λόγους για τους οποίους χρησιμοποιείται η μέθοδος K-fold cross validation.</li>
<li>Κατά τη χρήση της μεθόδου K-fold cross validation, ανά fold, να αναφέρετε πόσα examples χρησιμοποιούνται για την εκπαίδευση του μοντέλου και πόσα για το validation/testing.
<li>Δείτε <a href="https://en.wikipedia.org/wiki/Cross-validation_(statistics)">εδώ</a> μεθόδους που μπορεί να χρησιμοποιούν έναντι της μεθόδου K-fold cross validation. Να αναφέρετε μερικά πλεονεκτήματα και μειονεκτήματά τους.</li>
<li>Να αναλύσετε τα κριτήρια precision, recall και F1-score που εμφανίζονται στο classification report.</li>
<li>Σε ποιες κατηγορίες πετυχαίνουμε καλύτερα αποτελέσματα και σε ποιες χειρότερα; Γιατί πιστεύετε ότι παίρνουμε αυτά τα αποτελέσματα;</li>
</ul>

<li><i>Γιατί το LSTM είναι κατάλληλο μοντέλο για την επίλυση του συγκεκριμένου προβλήματος;</i></li>

Το LSTM διατηρεί ένα cell state το οποίο δίνει στο μοντέλο τη δυνατότητα να αποθηκεύει πληροφορία από προηγούμενα δεδομένα μιας ακολουθίας δεδομένων. Στο υπό μελέτη πρόβλημα, αξιοποιούνται 100,000 διευθύνσεις (<a href="https://github.com/chrmor/DGA_domains_dataset">data source</a>) από ένα dataset 675,000 domain names (διευθύνσεων), εκ των οποίων οι 50\% είναι Alexa domains, και τα υπόλοιπα 50\% έχουν παραχθεί από 25 διαφορετικές οικογένειες Domain Generation Algorithms (DGAs). Κάθε οικογένεια εκ των 25 έχει συνεισφέρει από 13,500 domains, ενώ από τους 25 DGA algortihms, οι 13 είναι time dependent και οι εναπομείναντες 12 time independent. Από το dataset αυτό χρησιμοποιούνται μόνο οι 100,000 πρώτες εγγραφές που περιέχουν περίπου 50,000 dga generated domain names και 50,000 legit domain names όπως αυτά έχουν παρθεί από το Alexa (legit domains).

Εφόσον κάθε domain name string μπορεί να θεωρηθεί ως μια ακολουθία χαρακτήρων (chars), τότε το LSTM μπορεί να διατηρεί μνήμη για την ακολουθία πριν κάνει την τελική πρόβλεψη. Μέσω του cell state, όπως αναφέρθηκε, μεταφέρεται παρελθοντική πληροφορία από την ακολουθία στο τελευταίο LSTM cell, όπου και γίνεται η τελική πρόβλεψη για την κατηγορία ενός domain name. Αυτή η λειτουργία του LSTM λέγεται seq2vec, όπου δηλαδή μια ακολουθία δεδομένων δίνεται ως είσοδος και στην έξοδο παίρνουμε ένα διάνυσμα, από το οποίο εξάγεται η κλάση στην οποία ταξινομείται ένα domain name (ακολουθία). Tα LSTM μπορούν να ανακαλύψουν, συνεπώς, συσχετίσεις μεταξύ των χαρακτήρων της κάθε ακολουθίας οδηγώντας σε καλύτερη ταξινόμηση τελικά.

<li><i>Τι είναι η λίστα ονομάτων Alexa (<a href="https://en.wikipedia.org/wiki/Alexa_Internet">info</a>), ονόματα της οποίας έχουν χρησιμοποιηθεί για την κατηγορία των καλόβουλων ονομάτων;</i></li>

Η Alexa ήταν μια Αμερικάνικη εταιρεία ανάλυσης διαδικτυακής κίνησης (web traffic). Ανήκε εξ ολοκλήρου στην Amazon και η υπηρεσία Alexa σταμάτησε τη λειτουργία της τον Μάιο του 2022. Παρείχε δεδομένα διαδικτυακής κίνησης, παγκόσμιες κατατάξεις και άλλες πληροφορίες για πάνω από 30 εκατομμύρια ιστότοπους. Στο υποσόνολο των 100,000 domain names που χρησιμοποιείται, περί τα 50,000 είναι καλόβουλα, υπαρκτά domain names (legit) και έχουν προέλθει από τη λίστα αυτών των 30 εκατομμυρίων websites που έκανε track η Alexa.

Αντίθετα, τα υπόλοιπα περίπου 50,000 domain names είναι μη υπαρκτά dga generated domain names.

<li><i>Να διαλέξετε δύο οικογένειες malware από την κατηγορία dga, δηλαδή την κατηγορία με τα κακόβουλα ονόματα που αντιστοιχούν σε malware. Να αναζητήσετε τι προβλήματα δημιουργούν τα malware αυτά, π.χ. υποκλοπή τραπεζικών κωδικών, έναρξη επιθέσεων DDoS, κλπ.</i></li>

Όπως αναφέραμε προηγουμένως, τα μη υπαρκτά/τεχνητά domain names προέρχονται από 25 οικογένειες DGA αλγορίθμων. Αυτές είναι οι εξής:

\begin{table}
 \begin{tabular}{||c ||}
 \hline
 \text{DGA family} \\
 \hline\hline
 gozi \\
 corebot \\
 ranbyus \\
 symmi \\
 emotet \\
 dircrypt \\
 matsnu \\
 simda \\
 fobber \\
 pushdo \\
 qadars \\
 kraken \\
 ramnit \\
 nymaim \\
 pykspa \\
 tinba \\
 murofet \\
 cryptolocker \\
 ramdo \\
 vawtrak \\
 conficker \\
 padcrypt \\
 rovnix \\
 suppobox \\
 necurs \\
 \hline
 \end{tabular}
\end{table}

Ενδεικτικά, θα περιγράψουμε τα προβλήματα που δημιουργούν 2 από αυτές τις οικογένεις malware, την kraken και την ......

Η **Kraken** είναι η πρώτη οικογένεια malware που χρησιμοποίησαν DGAs το 2008. Είναι network hacking spyware software που επιτίθεται στα Microsoft Windows και Apple Macintosh λειτουργικά συστήματα μέσω email και world wide web sites όπως μέσα κοινωνικής δικτύωσης. Οι συσκευές που γίνονται infected χρησιμοποιούνται συνήθως για να στέλνουν spam email messages.

Ουσιαστικά, όλη η λογική πίσω από την παραγωγή domain names (DGA) από κώδικα που τρέχει σε infected συσκευή, αφορά την αποφυγή να γίνουν blacklisted οι Command & Control Servers που μπορεί να χρησιμοποιούνται. Για παράδειγμα, ο DGA του kraken γεννά μέσω ενός seeded αλγορίθμου ένα πλήρες domain name με ένα τυχαίο string που αποτελείται από σύμφωνα και φωνήεντα που σχεδόν μιμούνται την αγγλική γραμματική και δομή των λέξεων λόγω shifting των χαρακτήρων. Όταν μια τυχαία λέξη λοιπόν παραχθεί τότε γίνεται append ένα domain suffix ώστε να δημιουργηθεί το τελικό domain name. Έπειτα μια IoT συσκευή κατά βάση προσπαθεί να συνδεθεί με τον command & control server για να λάβει το επόμενο σετ οδηγιών.

Η **Cryptolocker** malware οικογένεια, από την άλλη, πρόκειται για ένα trojan horse που μολύνει έναν υπολογιστή κι έπειτα αναζητά αρχεία προς κρυπτογράφηση, συμπεριλαμβανομένου αρχείων στον σληρό δίσκο και άλλα μέσα αποθήκευσης (πχ USB memory sticks). Επίσης, μπορεί να αναζητά αρχεία και φακέλους αποθηκευμένους στο cloud. Κάνουν infect μόνο στο Windows λειτουργικό και όχι Macintosh. Το malware όταν μολυνθεί ένας υπολογιστής εφαρμόζειμια μέθοδο encryption στα αρχεία που λέγεται assymetric και βασίζεται σε ένα δημόσιο και ένα ιδιωτικό κλειδί. Οι hackers κωδικοποιούν τα δεδομένα χρησιμοποιώντας το δημόσιο κλειδί, ωστόσο για την αποκρυπτογράφηση χρειάζεται και το μοναδικό ιδιωτικό κλειδί που κατέχουν. Το malware αυτό όταν κρυπτογραφήσει τα δεδομένα εμφανίζει παράθυρα που ενημερώνουν το χρήστη ότι τα δεδομένα θα καταστραφούν αν δεν πληρωθεί κάποιο χρηματικό ποσό για να λάβει το ιδιωτικό κλειδί αποκρυπτογράφησης.

<li><i>Να περιγράψετε σύντομα τα βήματα που ακολουθεί το παραπάνω πρόγραμμα για την επίλυση του προβλήματος.<i></li>

Αρχικά, διαβάζεται το πλήρες dataset και επιστρέφεται μια λίστα που περιέχει ζεύγη τιμών σε λίστες [domain_name_prefix, family], όπου το domain_name_prefix είναι το string που προηγείται της τελείας στο domain name και family μπορεί να παίρνει οποιαδήποτε από 26 τιμές (25 για τις οικογένειες DGA και άλλη μία η 'alexa'). Επίσης επιστρέφονται οι μοναδικοί χαρακτήρες (chars) που έχουν εμφανιστεί στα prefixes αυτά, ενώ υπολογίζεται και το μέγιστο μήκος prefix όπως και ένα dictionary με keys τις 26 κατηγορίες και values από 0 έως 25 για τις κατηγορίες αυτές.

Έπειτα, κρατούνται μόνο οι 100,000 πρώτες εγγραφές του dataset. Ύστερα ως χαρακτηριστικά λογίζονται οι μοναδικοί χαρακτήρες που εμφανίζονται στο dataset και τους γίνεται assigned ως value ένα index  (σε dict με keys τα chars αυτά και values τα indexes). Κατόπιν, το dataset μετατρέπεται/γίνεται tokenized, υπό την έννοια ότι κάθε στοιχείο του dataset μετασχηματίζεται και παίρνει αριθμητικές τιμές συν το label (κατηγορία από τις 26), ενώ προστίθεται και padding όπου χρειάζεται ώστε να έχουν ίδιο μήκος χαρακτηριστικών τα διάφορα στοιχεία του dataset. Ύστερα, χωρίζονται τα examples με τα χαρακτηριστικά και το label, και έπειτα ακολουθεί ένα 5-fold cross validation.

 Στο cross validation, για κάθε iteration της διαδικασίας, χωρίζεται το dataset σε X_train, y_train, X_test, y_test και δημιουργείται ένα μοντέλο LSTM, το οποίο εκπαιδεύται στα δεδομένα εκπαίδευσης για 5 εποχές με batch size 128 (δηλαδή 128 δειγματικά στοιχεία περνούν στο forward pass τη φορά πριν υπολογιστούν τα gradients και γίνει ένα update στα βάρη του δικτύου). Ύστερα το εκπαιδευμένο μοντέλο χρησιμοποιείται για πρόβλεψη ετικετών/κλάσεων στα δεδομένα του X_test και επιστρέφονται το classification report και το confusion matrix στο test fold.

<li></i>Ποιος είναι ο ρόλος του embedding layer και τι παραμέτρους δέχεται;</i></li>

Όπως αναφέρθηκε προηγουμένως, για να λειτουργήσει το δίκτυο LSTM απαιτείται φυσικά να δουλέψει με αριθμητικά διανύσματα. Έτσι, μετασχηματίσαμε τα prefix strings σε διανύσματα/λίστες με τιμές χαρακτηριστικών. Το embedding layers μετατρέπει με τη σειρά του φυσικούς (αριθμούς) σε dense διανύσματα με συγκεκριμένο μήκος/μέγεθος. Ουσιαστικά χρησιμοποιείται πριν εισαχθούν τα δεδομένα στο LSTM. Δέχεται ορισμένες παραμέτρους όπως το μέγιστο μήκος ενός string στα prefixes των domain name strings όπως και το επιθυμητό μήκος των διανυσμάτων εξόδου.

<li><i>Να αναφέρετε λόγους για τους οποίους χρησιμοποιείται η μέθοδος K-fold cross validation.</i></li>

Η μέθοδος k-fold cross validation είναι μια μέθοδος resampling που χρησιμοποιεί διαφορετικά κομμάτια των δεδομένων για να εκπαιδεύσει και να αποτιμήσει ένα μοντέλο σε διαφορετικά (k σε αριθμό) iterations. Χρησιμοποιείται κυρίως σε περιπτώσεις όπου πρωταρχικός στόχος είναι η πρόβλεψη και κανείς θέλει να εκτιμήσει την ακρίβεια την οποία το προβλεπτικό μοντέλο αναμένεται να έχει στην πράξη σε νέα δεδομένα. Η μέθοδος μπορεί επίσης να βοηθήσει στον εντοπισμό προβλημάτων όπως το overfitting ή το selection bias.

Με τη μέθοδο k-fold cross validation, εκπαιδεύονται ουσιαστικά k ταξινομητές της ίδιας κλάσης σε διαφορετικά k-1 folds κάθε φορά ως σύνολα εκπαίδευσης, και η αποτίμηση γίνεται στο k-οστό fold. Κάνοντας aggregate τις αποτιμήσεις των k αυτών μοντέλων πρακτικά κινούμαστε προς έναν μέσο όρο της απόδοσης της κλάσης αυτής του ταξινομητή στο πρόβλημα που εξετάζεται, απαλείφοντας εξαρτήσεις με μέρος των δεδομένων που θα μπορούσαν να οδηγούν σε τυχαία πολύ καλά ή πολύ άσχημα αποτελέσματα.

<li><i>Κατά τη χρήση της μεθόδου K-fold cross validation, ανά fold, να αναφέρετε πόσα examples χρησιμοποιούνται για την εκπαίδευση του μοντέλου και πόσα για το validation/testing</i>.</li>

Αν υποθέσουμε πως τα δειγματικά στοιχεία του συνόλου δεδομένων (δείγμα) είναι $N$ σε αριθμό, τότε για κάθε μια επανάληψη (ανά test fold) του cross validation, έχουμε:
- Ο αριθμός των δειγματικών στοιχείων ανά fold είναι $\frac{N}{k}$ αν υποθέσουμε πως διαιρείται το $Ν$ ακριβώς με το $k$.
- $k-1$ folds για εκπαίδευση, ήτοι $(k-1) \cdot \frac{N}{k} = \frac{k-1}{k} \cdot N$ δειγματικά στοιχεία εκπαίδευσης.
- 1 fold για αξιολόγηση, ήτοι $1 \cdot \frac{N}{k} = \frac{1}{k} \cdot N$ δειγματικά στοιχεία.

Όπως είναι εμφανές τα δειγματικά στοιχεία εκπαίδευσης συν τα δειγματικά στοιχεία αξιολόγησης αθροίζουν σε αριθμό στο $Ν$. Εν προκειμένω έχουμε $Ν=100,000$, οπότε για $k=5$ έχουμε $\frac{k-1}{k} \cdot N = \frac{5-1}{5} \cdot 100,000 = 80,000$ δειγματικά στοιχεία εκπαίδευσης και  $\frac{1}{k} \cdot N = \frac{1}{5} \cdot 100,000 = 20,000 $ δειγματικά στοιχεία αξιολόγησης για κάθε iteration/fold.

<li><i>Δείτε <a href="https://en.wikipedia.org/wiki/Cross-validation_(statistics)">εδώ</a> μεθόδους που μπορεί να χρησιμοποιούν έναντι της μεθόδου K-fold cross validation. Να αναφέρετε μερικά πλεονεκτήματα και μειονεκτήματά τους.</i></li>

Αρχικά, υπάρχουν δύο μέθοδοι cross-validation, οι **exhaustive**, και οι **non-exhaustive**.

Οι **exhaustive** μέθοδοι είναι cross-validation μέθοδοι που μαθαίνουν και εξετάζεται η επίδοσή τους με όλους τους πιθανούς τρόπους διαίρεσης/διαχωρισμού του αρχικού δείγματος σε training και validation sets. Εν αντιθέσει, οι **non-exhaustive** μέθοδοι δεν υπολογίζουν ή εξετάζουν όλους του πιθανούς τρόπους διαίρεσης/διαχωρισμού του δείγματος και αποτελούν προσεγγίσεις του leave-p-out cross-validation. Η μέθοδος k-fold cross-validation αποτελεί μια τέτοια μέθοδο (non-exhaustive).

Η μέθοδος leave-p-out cross-validation (LpO CV) πόυ αναφέρθηκε είναι exhaustive μέθοδος, και χρησιμοποιεί ένα σύνολο p παρατηρήσεων ως validation set και τις υπόλοιπες παρατηρήσεις ως το training set. Αυτό επαναλαμβάνεται για όλους τους τρόπους που μπορεί να χωριστεί το αρχικό σύνολο δεδομένων/δείγμα σε validation και training set υπό τη συνθήκη το validation set να περιέχει p παρατηρήσεις. Αν το δείγμα έχει μέγεθος n (συνολικός αριθμός παρατηρήσεων), τότε για $p>1$ πολύ εύκολα και όχι για τεράστιο n ο υπολογισμός μπορεί να μην είναι καν υλοποιήσιμος από υπολογιστική σκοπιά. Για παράδειγμα, αν $n=100$ και $p=20$, τότε θα χρειαστούν $C_p^n=C_{20}^{100}=\binom{100}{20} \approx 5.4 \cdot 10^{20}$ μοντέλα να εκπαυδευτούν με το training set και να γίνει αξιολόγηση τους πάνω στο test set ώστε να έχουμε εφαρμόσει την LpO CV μέθοδο. Αυτό είναι και το βασικό μειονέκτημα της μεθόδου και ένας από τους λόγους που χρησιμοποιούμε non-exhaustive μεθόδους όπως το k-fold cross validation. Για $p=1$, η προηγούμενη μέθοδος ανάγεται στην leave-one-out-cross-validation exhaustive μέθοδο, όπου το validation set είναι μια ακριβώς παρατήρηση. Όπως είναι εμφανές στην περίπτωση αυτή έχουμε $C_p^n=C_{1}^{100}=\binom{100}{1} = 100$ μοντέλα και εν γένει n μοντέλα για μέγεθος δείγματος n. Αν το n είναι σχετικά μικρό η μέθοδος αυτή μπορεί να εφαρμοστεί, ωστόσο για n μεγάλο, τα μειονεκτήματα είναι και πάλι τα ίδια με πριν. Άλλοι μέθοδοι από τις non-exhaustive που μπορούν να χρησιμοποιηθούν αντί της k-fold cross validation είναι η **holdout method** και η **repeated random sub-sampling validation** μέθοδος. Η holdout μέθοδος ουσιαστικά αφορά το διαχωρισμό των δεδομένων σε train και test sets, την εκπαίδευση στο train και την αξιολόγηση στο test set. Η απουσία multiple runs και averaging του κριτηρίου αξιολόγης όπως γίνεται στο k-fold cross validation μπορεί να οδηγήσει σε παραπλανητικά αποτελέσματα με τη χρήση της holdout μεθόδου.  Η repeated random sub-sampling validation μέθοδος (επίσης γνωστή ως Monte-carlo cross-validation) δημιουργεί τυχαία splits του dataset σε training και validation sets. Έτσι κάποιες παρατηρήσεις μπορεί να μην επιλεχθούν ποτέ ως μέρη του validation set ενώ άλλες παρατηρήσεις μπορεί να επιλεγούν παραπάνω από μία φορές. Με τη μέθοδο αυτή απομπλέκονται ο αριθμός επαναλήψεων της εκπαίδευσης και αξιολόγης του μοντέλου (k στην περίπτωση του k-fold cross validation) από τη σχέση αναφορικά με τον αριθμό δειγματικών στοιχείων που περιέχουν μεταξύ train και validation sets.

<li><i>Να αναλύσετε τα κριτήρια precision, recall και F1-score που εμφανίζονται στο classification report.</i></li>

Tα κριτήρια precision, recall και F1-score είναι μετρικές απόδοσης της ταξινόμησης.

<ul>
<li> Precision: Το precision εκφράζει τη δυνατότητα ενός ταξινομητή να ανακαλύπτει μόνο τα σχετικά data points. Απαντά στην ερώτηση "Τι ποσοστό των θετικά ταξινομημένων δειγματικών στοιχείων ήταν τελικά πραγματικά θετικά" και ορίζεται ως $Precision=\frac{TP}{TP+FP}$. Σε ένα παράδειγμα με 2 κλάσεις, έστω πρόβλεψη ύπαρξης καρκίνου (έστω ότι η κλάση ασθενή με καρκίνο είναι η θετική και χωρίς καρκίνο είναι η αρνητική), εάν ο ταξινομητής έχει precision ίσο με 0.5, τότε όταν προβλέπει ότι ένας ασθενής έχει καρκίνο τότε έχει 50% πιθανότητα να έχει κάνει σωστή πρόβλεψη. </li>
<li> Recall: Το recall εκφράζει τον αριθμό των data points που ένας ταξινομητής σωστά αναγνώρισε ότι ανήκουν στη θετική κλάση από το συνολικό αριθμό θετικών δειγματικών στοιχείων. Απαντά στην ερώτηση "Τι ποσοστό των πραγματικά θετικών δειγμάτων ταξινομήθηκαν σωστά" και ορίζεται ως $Recall=\frac{TP}{TP+FN}$. Στο ίδιο παράδειγμα με τον καρκίνο, αν το μοντέλο μας έχει μετρική αξιολόγησης recall ίση με 0.2 τότε αναγνωρίζει σωστά το 20% των ασθενών με καρκίνο.</li>
<li> F1-score: Είναι οαρμονικός μέσος όρος των precision και recall, δηλαδή $F1-score = 2 \cdot frac{Precision \cdot Recall}{Precision + Recall}$. Με το F1-score συνδυάζονται πρακτικά και οι δύο μετρικές σε μία.</li>
</ul>
Αξίζει να σημειώσουμε και το accuracy metric, όπου $Accuracy =  \frac{TP+TN}{TP+TN+FP+FN}$ ως μετρική αξιολόγησης ενός ταξινομητή. Ωστόσο, ειδικά σε περιπτώσεις που ειναι οι κλάσεις είναι μη ισορροπημένες, η μετρική αυτή είναι προβληματική πιθανότατα καθώς μπορεί να δίνει τη λανθασμένη εντύπωση πως ο ταξινομητής γενικεύει ικανοποιητικά πετυχαίνοντας καλά σκορ στη μετρική αυτή του accuracy, χωρίς αυτό να αντικατοπτρίζει την πραγματικότητα. Σε τέτοιες ειδικά περιπτώσεις επιβάλλεται η χρήση και άλλων μετρικών όπως το precision, recall, f1-score.


<li><i>Σε ποιες κατηγορίες πετυχαίνουμε καλύτερα αποτελέσματα και σε ποιες χειρότερα; Γιατί πιστεύετε ότι παίρνουμε αυτά τα αποτελέσματα;</i></li>

Από το classification report στα 5 folds παρατηρούμε ότι η κατηγορία Alexa που αφορά τα πραγματικά domain names και περίπου τα μισά δειγματικά στοιχεία του test set πετυχαίνει καλά classification metrics, ενώ από τις υπόλοιπες 25 κατηγορίες με DGA generated domain names, άλλες πάνε καλύτερα και κάποιες έχουν πολύ χαμηλά scores. Γενικά στα 5 folds τα αποτελέσματα είναι πανομοιότυπα χωρίς σημαντικές αποκλίσεις. Κάποιες κλάσεις που παρατηρούμε ότι επιστρέφουν σχετικά χαμηλές μετρικές αξιολόγησης είναι οι εξής: dircrypt, kraken, ramnit και suppobox. Στην κλάση alexa και στην corebot εν αντιθέσει, η ταξινόμηση επιστρέφει ικανοποιητικά αποτελέσματα.

Ένας από τους λόγους της κακής απόδοσης σε κάποιες από τις κατηγορίες είναι η ανισορροπία των κλασεων, όπου έχουμε πολύ περισσότερα δειγματικά στοιχεία από την κλάση Alexa. Αυτό σημαίνει ότι ενδεχομένως ο ταξινομητής είναι biased ως προς την επιλογή/ταξινόμηση ενός δειγματικού στοιχείου στην κλάση alexa. Από το confusion matrix που εκτυπώνεται μπορεί κανείς για παράδειγμα να δει αυτή τη συμπεριφορά στην κατηγορία suppobox, όπου πολλά δειγματικά στοιχεία που πραγματικά ανήκουν στην κλάση αυτή, τελικώς ταξινομήθηκαν (λανθασμένα) στην κλάση alexa. Σε άλλες περιπτώσεις κλάσεω που τα δειγματικά στοιχεία προσομοιάζουν, ο ταξινομητής μπερδεύεται και ταξινομεί δειγματικά στοιχεία από τη μία στην άλλη.
